### Analysis of the DataSet

In [52]:
import pandas as pd
import numpy as np
import networkx as nx
import csv

### Preprocessing

In [53]:
def getSize(fileName):
    dataFrame = pd.read_csv(fileName,delimiter=',')
    traderList = list(dataFrame['Seller'])
    traderList.extend(list(dataFrame['Buyer']))
    traderSet = set(traderList)
    traderSize = len(traderSet)
    return traderSize

### Undirected Graph generation

In [54]:
# total transaction amount between them is the weight
def getUndirectedGraph(fileName):
#     K represents k nearest neighbors
    dataFrame = pd.read_csv(fileName, delimiter =',')
    nodeSize = getSize(fileName)
    myGraph = nx.Graph()
#     Added nodes to the Graph
    myGraph.add_nodes_from(list(range(1, nodeSize+1)))
#    Adding edges with weights to the Graph
    buyer = []
    seller = []
    trader = []
    for ind in dataFrame.index:
        u = dataFrame['Seller'][ind]
        v = dataFrame['Buyer'][ind]
        seller.append(u)
        buyer.append(v)
    
    vSeller = np.zeros(nodeSize+1)
    vBuyer = np.zeros(nodeSize+1)

    for x in seller:
        vSeller[x] =1 
    for x in buyer:
        vBuyer[x] = 1 

    for ind in dataFrame.index:
        u = dataFrame['Seller'][ind]
        v = dataFrame['Buyer'][ind]
        w = dataFrame['Amt'][ind]
        if (vSeller[u]==1) and (vBuyer[u]==1) and (vSeller[v]==1) and (vBuyer[v]==1) and(u!=v):
            trader.append(u)
            trader.append(v)
            if u in list(myGraph.adj[v]):
                myGraph[u][v]['weight']+=w
            else :
                myGraph.add_edge(u, v, weight = w)
    return (myGraph,list(set(trader)))

(undirectedGraph,nodeSet) = getUndirectedGraph('dataset.csv')

### Directed Graph generation

In [97]:
def getDirectedGraph(fileName):
    dataFrame = pd.read_csv(fileName, delimiter =',')
    nodeSize = getSize(fileName)
    myGraph = nx.DiGraph()
#     Added nodes to the Graph
    myGraph.add_nodes_from(list(range(1, nodeSize+1)))
#    Adding edges with weights to the Graph 
    for ind in dataFrame.index:
        u = dataFrame['Seller'][ind]
        v = dataFrame['Buyer'][ind]
        w = dataFrame['Amt'][ind]
        if u in list(myGraph.adj[v]):
            myGraph[v][u]['weight']+=w
        else :
            myGraph.add_edge(v, u, weight = w)
    return myGraph

DirectedGraph = getDirectedGraph('dataset.csv')

### Shared nearest neighbour Clustering Algorithm

### K-Nearest Neighbour

In [56]:
%%time
def kNear(graph, K):
    Dict = {}
    for node in nodeSet:
        neighbors = np.array(sorted(graph[node].items(), key=lambda e: e[1]["weight"], reverse=True))
        if(neighbors.shape[0]<K):
            neighborList = [x[0] for x in neighbors]
        else : 
            neighbors = neighbors[:K]
            neighborList = [x[0] for x in neighbors]
        Dict[node]=neighborList
    return Dict

knnGraph = kNear(undirectedGraph, 6)

CPU times: user 202 ms, sys: 5.59 ms, total: 208 ms
Wall time: 216 ms


### Union find algorithm

In [136]:
nodeSize = getSize('dataset.csv')
clusterId = np.array(list(range(nodeSize+1)))
clusterSize = np.ones(nodeSize, dtype = int)
def parent(u):
    while clusterId[u]!=u:
        u = clusterId[u]
    return u
def common(u, v):
    neighborU = knnGraph[u]
    neighborV = knnGraph[v]
    return len(list(set(neighborU)&set(neighborV)))
def union(u, v):
    x = parent(u)
    y = parent(v)
    global clusterId
    global clusterSize
    if clusterSize[x]>clusterSize[y]:
        clusterId[y] = x
        clusterSize[x]+=clusterSize[y]
    else :
        clusterId[x] = y 
        clusterSize[y]+=clusterSize[x]

### Shared Nearest Neighbor Algorithm

In [137]:
%%time
kt = 2
for u in nodeSet:
    for v in nodeSet:
        x = parent(u)
        y = parent(v)
        if x!= y :
            if (u in knnGraph[v]) and (v in knnGraph[u]):
                if common(u,v)>=kt:
                    union(u,v)
# union find
# set - > all nodes, each node as it's own cluster
# for every (u,v) in G : 
# if v not in C(u) && if k(u) && k(v)>= kt && u->knn(V) && v->knn(u):
# remove c(u), c(v) 
# add c(u)union c(v) to set S

CPU times: user 1min 4s, sys: 243 ms, total: 1min 4s
Wall time: 1min 5s


### Final dictionary of Shared nearest neighbor

In [138]:
SNNClust = {}
vis = np.zeros(nodeSize+1, dtype = int)
for node in range(1, nodeSize+1):
    currId = parent(node)
    if vis[currId]==0:
        SNNClust[currId]=[node]
    else:
        SNNClust[currId].append(node)
    vis[currId] = 1
print(len(SNNClust))

32371


In [140]:
Dict = {}
for x in SNNClust:
    if len(SNNClust[x])>2:
        Dict[x]=SNNClust[x]

In [141]:
w = csv.writer(open("Shared_Nearest_Neighbor.csv", "w"))
for key, val in Dict.items():
    w.writerow([key, val])

### Mutual Nearest Neighbour

In [164]:
knnGraph = kNear(DirectedGraph, 6)
DictofClusters = {}
for i in nodeSet:
    DictofClusters[i] = [i]

In [165]:
def mnvForNodes(u, v):
    listU = knnGraph[u]
    listV = knnGraph[v]
    rankOfVinU = 100
    rankOfUinV = 100
    
    for i in range(len(listU)):
        if listU[i]==v:
            rankOfVinU = i+1
            break
            
    for i in range(len(listV)):
        if listV[i]==u:
            rankOfUinV = i+1
            break

    return rankOfUinV+rankOfVinU

In [166]:
def mnvForClusters(c1, c2):
    cluster1 = DictofClusters[c1]
    cluster2 = DictofClusters[c2]
    s = 0
    for u in cluster1:
        for v in cluster2:
            val = mnvForNodes(u, v)
            s = s + val
    length = len(cluster1) * len(cluster2)
    return s/length

In [167]:
%%time
mnvSet = set([])
# for all pairs of clusters, find the mnv and insert into the set
for c1 in nodeSet:
    for c2 in nodeSet:
        if c1!=c2:
            currVal = mnvForClusters(c1, c2)
            mnvSet.add((currVal, c1, c2))

CPU times: user 2min 36s, sys: 5.83 s, total: 2min 42s
Wall time: 2min 45s


In [168]:
%%time
def mutual_NN_neighbour(numofClusters, maxMNV):
    global DictofClusters
    global mnvSet
    totalClusters = len(DictofClusters)
    print("All pair combination calculations done")
#     keep iterating while you didn't get required number of clusters
    cnt = 0 
    while totalClusters>numofClusters:
        minCluster = min(mnvSet)
        minU = minCluster[1]
        minV = minCluster[2]
        minNodes = [minU, minV]
#         remove all pairs which are either of minU,minV
        for c1 in DictofClusters:
            for node in minNodes:
                val = mnvForClusters(c1, node)
                pair1 = (val, c1, node)
                pair2 = (val, node, c1)
                if pair1 in mnvSet:
                    mnvSet.remove(pair1)
                if pair2 in mnvSet:
                    mnvSet.remove(pair2)
#       remove two clusters and combine them into one
        for v in DictofClusters[minV]:
            DictofClusters[minU].append(v)
        del DictofClusters[minV]
#         insert all the mnvs of the new clusters with the clusters existing
        for c1 in DictofClusters:
            if minU!=c1:
                currVal = mnvForClusters(c1, minU)
                mnvSet.add((currVal, c1, minU))
        print("Iteration ", cnt+1, "completed")
        cnt+=1
        totalClusters -=1
#         remove all the elements computing with c1, c2
# do the manipulations in the dictionary
# calculate the clustered one with every other cluster
# and push it into the set

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 14.1 µs


In [175]:
%%time
mutual_NN_neighbour(len(nodeSet)-500, 100)

All pair combination calculations done
Iteration  1 completed
Iteration  2 completed
Iteration  3 completed
Iteration  4 completed
Iteration  5 completed
Iteration  6 completed
Iteration  7 completed
Iteration  8 completed
Iteration  9 completed
Iteration  10 completed
Iteration  11 completed
Iteration  12 completed
Iteration  13 completed
Iteration  14 completed
Iteration  15 completed
Iteration  16 completed
Iteration  17 completed
Iteration  18 completed
Iteration  19 completed
Iteration  20 completed
Iteration  21 completed
Iteration  22 completed
Iteration  23 completed
Iteration  24 completed
Iteration  25 completed
Iteration  26 completed
Iteration  27 completed
Iteration  28 completed
Iteration  29 completed
Iteration  30 completed
Iteration  31 completed
Iteration  32 completed
Iteration  33 completed
Iteration  34 completed
Iteration  35 completed
Iteration  36 completed
Iteration  37 completed
Iteration  38 completed
Iteration  39 completed
Iteration  40 completed
Iteration 

Iteration  332 completed
Iteration  333 completed
Iteration  334 completed
Iteration  335 completed
Iteration  336 completed
Iteration  337 completed
Iteration  338 completed
Iteration  339 completed
Iteration  340 completed
Iteration  341 completed
Iteration  342 completed
Iteration  343 completed
Iteration  344 completed
Iteration  345 completed
Iteration  346 completed
Iteration  347 completed
Iteration  348 completed
Iteration  349 completed
Iteration  350 completed
Iteration  351 completed
Iteration  352 completed
Iteration  353 completed
Iteration  354 completed
Iteration  355 completed
Iteration  356 completed
Iteration  357 completed
Iteration  358 completed
Iteration  359 completed
Iteration  360 completed
Iteration  361 completed
Iteration  362 completed
Iteration  363 completed
Iteration  364 completed
Iteration  365 completed
Iteration  366 completed
Iteration  367 completed
Iteration  368 completed
Iteration  369 completed
Iteration  370 completed
Iteration  371 completed


In [176]:
Dict = {}
for x in DictofClusters:
    if len(DictofClusters[x])>1:
        Dict[x] = DictofClusters[x]
w = csv.writer(open("Mutual_Nearest_Neighbor.csv", "w"))
for key, val in Dict.items():
    w.writerow([key, val])